# Weaknesses of Linear Layers

Linear layer is a `fully connected layer` meaning that every neuron in a given layer is connected to every neuron in the next layer.  
This has two key implications:
- It results in a `LOT of parameters`.
- `The order of our features doesn’t matter.`

Consider the simple image and fully connected network below:

선형 계층(Linear layer)은 '완전 연결 계층(fully connected layer)'으로, 한 계층의 모든 뉴런이 다음 계층의 모든 뉴런과 연결됩니다. 이러한 특성에는 두 가지 주요한 영향이 있습니다.

많은 파라미터 생성: 완전 연결 계층은 많은 파라미터를 생성합니다. 각 뉴런 간의 모든 연결은 학습해야 하는 가중치로 이어지기 때문에, 입력 및 출력의 크기에 따라 파라미터 수가 기하급수적으로 증가할 수 있습니다.

특징의 순서 무시: 완전 연결 계층은 입력 특성의 순서를 고려하지 않습니다. 각 입력 특성이 모든 뉴런에 동등하게 전달되므로, 특성들 간의 위치나 순서적인 관계가 고려되지 않고 모든 입력이 동일하게 처리됩니다.

간단한 이미지와 완전 연결 네트워크를 고려하여 이러한 선형 계층의 한계를 이해해보겠습니다.

<img src="img/fully_connected_parameters.png" width=700>

<span style="color:red;font-size:26px"> What happens for 256x256 image? <br>It is not feasible to process larger images with linear layers!!</span> 

# Convolutions for Images
In a convolutional layer, an input array and a correlation kernel array are combined to produce an output array through a cross-correlation operation.  
In our example, the input is a two-dimensional array with a shape of $4 \times 4$ or (4, 4).  
The height and width of the kernel array are both 2.  
Common names for this array in the deep learning research community include `*kernel* and *filter*`.  
The shape of the kernel window (also known as the convolution window) is given precisely by the height and width of the kernel

합성곱(Convolutional) 계층에서는 입력 배열과 상관 관계(correlation) 커널 배열을 교차 상관(correlation) 연산을 통해 출력 배열을 생성합니다.
우리의 예에서, 입력은 $4 \times 4$ 또는 (4, 4) 모양의 이차원 배열입니다.
커널 배열의 높이와 너비는 모두 2입니다.
딥 러닝 연구 커뮤니티에서 이러한 배열을 일반적으로 *커널* 또는 *필터*라고 부릅니다.
커널 창의 모양(합성곱 윈도우로도 알려짐)은 커널의 높이와 너비로 정확하게 결정됩니다.
<span style="color:yellow;font-size:26">($2 \times 2$).


<img src="img/cnn.gif" width=700> <br>


<img src="img/correlation.svg" width=700>


The shaded portions are the first output element and the input and kernel array elements used in its computation: $0\times0+1\times1+3\times2+4\times3=19$. 

Here, the output array has a height of 2 and width of 2 and the four elements are derived from the two-dimensional cross-correlation operation:

음영 처리된 부분은 첫 번째 출력 요소와 해당 계산에 사용된 입력 및 커널 배열 요소입니다: $0\times0+1\times1+3\times2+4\times3=19$.

여기서 출력 배열은 높이가 2이고 너비가 2이며, 이 네 요소는 이차원 교차 상관(correlation) 연산에서 파생됩니다:

$$
0\times0+1\times1+3\times2+4\times3=19,\\
1\times0+2\times1+4\times2+5\times3=25,\\
3\times0+4\times1+6\times2+7\times3=37,\\
4\times0+5\times1+7\times2+8\times3=43.
$$

We implement the above process in the `corr2d` function.
It accepts the input array `X` with the kernel array `K`
and outputs the array `Y`.

우리는 위의 과정을 corr2d 함수로 구현합니다.
이 함수는 입력 배열 X와 커널 배열 K를 입력으로 받아 배열 Y를 출력합니다.

In [1]:
import torch
from torch import nn

def corr2d(X, K):
    # 커널의 높이와 너비
    h, w = K.shape
    # 출력 배열 초기화
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))

    # 2D 교차 상관 연산을 위한 반복문
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            # 교차 상관 연산 수행 및 결과 저장
            Y[i, j] = (X[i: i + h, j: j + w] * K).sum()
    return Y

# 입력 배열과 커널 배열 정의
X = torch.Tensor([[0, 1, 2], [3, 4, 5], [6, 7, 8]])
K = torch.Tensor([[0, 1], [2, 3]])

# 예제로 함수 호출 및 실행
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

## Convolutional Layers of Pytorch

A convolutional layer cross-correlates the input and kernels
and adds a scalar bias to produce an output.
The parameters of the convolutional layer
are precisely the values that constitute the kernel and the scalar bias.
When training the models based on convolutional layers,
we typically initialize the kernels randomly,
just as we would with a fully-connected layer.

We are now ready to implement a two-dimensional convolutional layer
based on the `corr2d` function defined above.
In the `__init__` constructor function,
we declare `weight` and `bias` as the two model parameters.
The forward computation function `forward`
calls the `corr2d` function and adds the bias.
As with $h \times w$ cross-correlation
we also refer to convolutional layers
as $h \times w$ convolutions.

합성곱(convolutional) 계층은 입력과 커널을 교차 상관하고, 스칼라 편향을 추가하여 출력을 생성합니다. 합성곱 계층의 매개변수는 정확히 커널과 스칼라 편향으로 이루어진 값들입니다. 합성곱 계층을 기반으로 하는 모델을 훈련할 때, 일반적으로 커널을 무작위로 초기화합니다. 이는 완전 연결 계층에서의 초기화와 유사합니다.

이제 앞서 정의한 corr2d 함수를 기반으로 2차원 합성곱(convolutional) 계층을 구현할 준비가 되었습니다. __init__ 생성자 함수에서 weight와 bias를 두 개의 모델 매개변수로 선언합니다. 순전파 연산 함수인 forward는 corr2d 함수를 호출하고 편향을 더합니다. $h \times w$ 교차 상관과 마찬가지로 우리는 합성곱 계층을 $h \times w$ 합성곱으로도 부릅니다.

In [2]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size, **kwargs):
        super(Conv2D, self).__init__(**kwargs)
        # 커널과 편향 초기화
        self.weight = torch.rand(kernel_size, dtype=torch.float32, requires_grad=True)
        self.bias = torch.zeros((1,), dtype=torch.float32, requires_grad=True)

    def forward(self, x):        
        # corr2d 함수와 편향을 더한 결과 반환
        return corr2d(x, self.weight) + self.bias


### General Convolution vs Dilated Convolution
<table>
  <tbody><tr>
    <td><a target="_blank" rel="noopener noreferrer" href="./img/no_padding_no_strides.gif"><img width="300px" src="./img/no_padding_no_strides.gif" data-animated-image="" style="max-width: 100%;"></a></td>
    <td><a target="_blank" rel="noopener noreferrer" href="./img/arbitrary_padding_no_strides.gif"><img width="300px" src="./img/arbitrary_padding_no_strides.gif" data-animated-image="" style="max-width: 100%;"></a></td>
    <td><a target="_blank" rel="noopener noreferrer" href="./img/same_padding_no_strides.gif"><img width="300px" src="./img/same_padding_no_strides.gif" data-animated-image="" style="max-width: 100%;"></a></td>
    <td><a target="_blank" rel="noopener noreferrer" href="./img/full_padding_no_strides.gif"><img width="300px" src="./img/full_padding_no_strides.gif" data-animated-image="" style="max-width: 100%;"></a></td>
  </tr>
  <tr>
    <td>No padding, no strides</td>
    <td>Arbitrary padding, no strides</td>
    <td>Half padding, no strides</td>
    <td>Full padding, no strides</td>
  </tr>
  <tr>
    <td><a target="_blank" rel="noopener noreferrer" href="./img/no_padding_strides.gif"><img width="300px" src="./img/no_padding_strides.gif" data-animated-image="" style="max-width: 100%;"></a></td>
    <td><a target="_blank" rel="noopener noreferrer" href="./img/padding_strides.gif"><img width="300px" src="./img/padding_strides.gif" data-animated-image="" style="max-width: 100%;"></a></td>
    <td><a target="_blank" rel="noopener noreferrer" href="./img/padding_strides_odd.gif"><img width="300px" src="./img/padding_strides_odd.gif" data-animated-image="" style="max-width: 100%;"></a></td>
    <td><a target="_blank" rel="noopener noreferrer" href="./img/conv_dilated.gif"><img width="300px" src="./img/conv_dilated.gif" data-animated-image="" style="max-width: 100%;"></a></td>
  </tr>
  <tr>
    <td>No padding, strides</td>
    <td>Padding, strides</td>
    <td>Padding, strides (odd)</td>
    <td><span style="color:yellow"> Dilated Convolution</td>
  </tr>
</tbody></table>

- Kernel Size : kernel size는 convolution의 시야(view)를 결정. 보통 2D에서 3x3 사용.
- Stride : 이미지를 횡단할 때 커널의 스텝 사이즈. 기본값은 1이지만 보통 Max Pooling과 비슷하게 이미지를 다운샘플링하기 위해 2도 사용.
- Padding : Padding은 샘플 테두리 처리 방법. 패딩된 Convolution은 input과 동일한 output 차원을 유지하는 반면, 패딩되지 않은 Convolution은 커널이 1보다 큰 경우 테두리의 일부가 사라짐.
- Input & Output Channels : Convolution layer는 Input 채널의 특정 수(I)를 받아 output 채널의 특정 수(O)로 계산.
- The number of Parameters: IxOxK로 계산. K는 커널의 수.

## Output size of 2D-Convolution Layer
`Conv2d with Stride=2 and padding`=$\frac{kernel}{2}$ means `downsampling` by $\frac{1}{2}$.

<img src="./img/conv_size.png" width=800>

In [1]:
import torch
import torch.nn as nn
import numpy as np

# 입력 및 출력 채널, dilation, 커널 크기, 스트라이드, 패딩, 이미지의 높이와 너비 정의
in_channels = 1
out_channels = 10
dilation = (1, 1)
kernel_size = (5, 5)
stride = (2, 2)
padding = (2, 2)
h, w = 32, 46

# 합성곱 연산 후 출력 크기 계산 함수 정의
def cal_conv_out(h, padding, kernel_size, stride, dilation=1):
    size = int(np.floor((h + 2 * padding - dilation * (kernel_size - 1) - 1) / stride + 1))
    return size 

# Conv2d 레이어 초기화 및 입력 데이터 생성
conv = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding, dilation=dilation)
x = torch.ones((1, in_channels, h, w))
y = conv(x)

# Conv2d 레이어로 계산된 출력 크기
output_h = cal_conv_out(h, padding[0], kernel_size[0], stride[0], dilation[0])
output_w = cal_conv_out(w, padding[1], kernel_size[1], stride[1], dilation[1])
print(x.shape, y.shape, f'[{output_h},{output_w}]')

# Average pooling 적용 후 출력 크기
pool = torch.nn.AvgPool2d(kernel_size=kernel_size, stride=stride, padding=padding)
y = pool(y)
print(y.shape)

# 두 번째 Conv2d 레이어 적용 후 출력 크기
conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding, dilation=dilation)
y = conv2(y)
print(y.shape)


torch.Size([1, 1, 32, 46]) torch.Size([1, 10, 16, 23]) [16,23]
torch.Size([1, 10, 8, 12])
torch.Size([1, 10, 4, 6])


c:\Users\user\AppData\Local\Programs\Python\Python38\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
# 입력 텐서 생성: 배치 크기 10, 채널 1, 높이 28, 너비 28인 텐서 생성
x = torch.ones((10, 1, 28, 28))

# Convolutional 레이어 생성: 입력 채널 1, 출력 채널 10, 커널 크기 5x5, 스트라이드 2, 패딩 2
conv2d = nn.Conv2d(1, 10, kernel_size=5, stride=2, padding=2)

# Convolutional 레이어를 통과한 출력
y = conv2d(x)
print(y.shape)

# Transposed Convolutional 레이어 생성: 입력 채널 10, 출력 채널 1, 커널 크기 5x5, 스트라이드 2, 패딩 2, 출력 패딩 1
convTrans2d = nn.ConvTranspose2d(10, 1, kernel_size=5, stride=2, padding=2, output_padding=1)

# Transposed Convolutional 레이어를 통과한 출력
y = convTrans2d(y)
print(y.shape)


torch.Size([10, 10, 14, 14])
torch.Size([10, 1, 28, 28])


## Transposed Convolutional Layers of Pytorch

<img src="./img/convTransposed2D.png" width=1000>

`ConvTranspose2d with Stride=2 and padding`=$\frac{kernel}{2}$ means `upsampling` by 2

In [24]:
import torch
import torch.nn as nn
import numpy as np

in_channels = 1
out_channels = 10

dilation = (1, 1)
kernel_size = (3, 3)
stride = (2, 2)
padding = (1, 1)
output_padding = (1, 1)
h, w = 16, 23

def cal_conv_out(h, padding, kernel_size, stride, dilation, output_padding):
    # Transposed Convolution 연산을 통해 출력 크기 계산
    size = (h - 1) * stride - 2 * padding + dilation * (kernel_size - 1) + output_padding + 1
    return size 

# Transposed Convolution 레이어 생성
conv = nn.ConvTranspose2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding, 
                          output_padding=output_padding, dilation=dilation)

# 입력 데이터 생성: 크기가 (1, in_channels, h, w)인 텐서
x = torch.ones((1, in_channels, h, w))

# Transposed Convolution 레이어 통과한 출력 계산
y = conv(x)

# 계산된 출력의 높이와 너비
output_h = cal_conv_out(h, padding[0], kernel_size[0], stride[0], dilation[0], output_padding[0])
output_w = cal_conv_out(w, padding[1], kernel_size[1], stride[1], dilation[1], output_padding[1])

# 입력과 출력의 형태, 출력의 높이와 너비 출력
print(x.shape, y.shape, f'[{output_h}, {output_w}]')

torch.Size([1, 1, 16, 23]) torch.Size([1, 10, 32, 46]) [32,46]


### Multi-channel convolution

<img src = 'img/torch_conv2d.png' width='800'>

## Output size of 1D-Convolution Layer
`Conv2d with Stride=2 and padding`=$\frac{kernel}{2}$ means `downsampling` by $\frac{1}{2}$.

<img src="./img/conv1d_shape.png" width=900>

In [4]:
import torch
import torch.nn as nn
import numpy as np

# 입력 채널, 출력 채널, dilation, 커널 크기, 스트라이드, 패딩 설정
in_channels = 1
out_channels = 10
dilation = 1
kernel_size = 3
stride = 2
padding = 1

# 입력 데이터 크기 설정
input_size = 10

# Convolution 연산 후 출력 크기 계산 함수
def cal_conv_out(input_size, padding, kernel_size, stride, dilation):
    size = int(np.floor((input_size + 2 * padding - dilation * (kernel_size - 1) - 1) / stride + 1))
    return size 

# Conv1d 레이어 생성
conv = nn.Conv1d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding, 
                 dilation=dilation)

# 입력 데이터 생성: 배치 크기 1, 입력 채널, 데이터 크기
x = torch.ones((1, in_channels, input_size))

# Conv1d 레이어를 통과한 출력 계산
y = conv(x)

# Convolution 연산을 통해 계산된 출력의 크기
output_size = cal_conv_out(input_size, padding, kernel_size, stride, dilation)
print(x.shape, y.shape, f'[{output_size}]')


torch.Size([1, 1, 10]) torch.Size([1, 10, 5]) [5]


### ConvTranspose1d

In [8]:
in_channels= 1  # 입력 데이터의 채널 수
out_channels= 10  # 출력 데이터의 채널 수
dilation = 1  # 실제 필터 값 사이의 간격을 결정하는 요소
kernel_size = 3  # 컨볼루션 커널(필터)의 크기
stride = 2  # 컨볼루션 연산 시 이동하는 보폭
padding = 1  # 입력 데이터 주변을 둘러싸는 0 값의 개수
output_padding = 1  # 출력 데이터 주변을 둘러싸는 0 값의 개수
input_size = 10  # 입력 데이터의 크기

# 컨볼루션 연산 후의 출력 크기를 계산하는 함수 정의
def cal_conv_out(input_size, padding, kernel_size, stride, dilation, output_padding):
        size = (input_size-1)*stride - 2*padding + dilation*(kernel_size-1) + output_padding + 1
        return size 

# nn.ConvTranspose1d 모듈을 사용하여 컨볼루션 레이어 정의
conv = nn.ConvTranspose1d(in_channels, out_channels, kernel_size= kernel_size, stride=stride, padding= padding, 
                          output_padding=output_padding,  dilation=dilation)

x = torch.ones((1, in_channels, input_size))  # 입력 데이터 생성
y = conv(x)  # 입력 데이터를 컨볼루션 레이어에 통과시켜 출력 데이터 생성

# 실제 출력 크기 계산
output_size = cal_conv_out(input_size, padding, kernel_size, stride, dilation, output_padding)

# 입력, 출력 데이터의 형태 및 실제 출력 크기 출력
print(x.shape, y.shape, f'[{output_size}]')

torch.Size([1, 1, 10]) torch.Size([1, 10, 20]) [20]
